In [ ]:
from jp_doodle import dual_canvas
from IPython.display import display
from datetime import datetime

In [ ]:
demo = dual_canvas.DualCanvasWidget(width=820, height=820, font='italic 50px Courier')
txt1 = demo.text(10,10,"00:00.000",color='red', name=True)
txt2 = demo.text(10,110,"00:00.000",color='blue', name=True)
txt3 = demo.text(10,210,"00:00.000",color='orange', name=True)
demo.rect(0,0,300,100,color='green')
demo.rect(0,200,300,100,color='green')
demo.fit()
demo.debugging_display()

In [ ]:
txts = [txt1,txt2,txt3]
start = datetime.now()

def switch(s=start):
    from datetime import datetime
    global txts
    [A,B,C] = txts
#     print(A.name,B.name,C.name)
    elap = datetime.now() - s
    sec = elap.seconds 
    mili = int(elap.microseconds/1000)
    minute = int(sec/60)
    A.transition(y=110,seconds_duration=0.8,text=str(minute)+':'+str(sec%60)+'.'+str(mili))
    B.transition(y=210,seconds_duration=0.8)
    C.change(y=10)#,seconds_duration=0.001)
    a, b, c = C, A, B
#     print(a.name,b.name,c.name)
    txts = [a, b, c]

In [ ]:
switch(start)

In [ ]:
demo.js_init("""
     element.setTimeout = function(callback, ms) {
         window.setTimeout(callback, ms);
     };
""")

def repeat_switch(ms=1000):
    switch()
    
    demo.element.setTimeout(repeat_switch, ms)

demo.element.setTimeout(repeat_switch, 1000)

In [ ]:
demo.js_debug()

_____

In [ ]:

from jp_doodle import dual_canvas
from IPython.display import display
from time import sleep
from datetime import datetime

class TimeUnit:
    
    init_txt = "00"
    dur = 0.6
    
    def __init__(self, demo):
        self.demo = demo
        color = self.color
        x = self.x
        text = self.init_txt
        txt1 = demo.text(x,10,text,color=color, name=True)
        txt2 = demo.text(x,110,text,color=color, name=True)
        txt3 = demo.text(x,210,text,color=color, name=True)
        self.txts = [txt1, txt2, txt3]
        self.start = datetime.now()
        
    def repeat_timing(self):
        self.timing()
        self.demo.element.setTimeout(self.repeat_timing, self.ms)
        
    def timing(self):
        [a, b, c] = self.txts
        txt = self.elapsed()
        a.transition(y=110, seconds_duration=self.dur, text='%02d' %txt)
        c.change(y=10)
        b.transition(y=210,seconds_duration=self.dur)
        self.txts = [c, a, b]
    
        
class Minutes(TimeUnit):
    x = 10
    color = "red"
    ms = 60000
    
    def elapsed(self):
        elap = datetime.now() - self.start
        sec = elap.seconds
        return int(sec/60)
        
    
class Seconds(TimeUnit):
    x = 110
    color = "blue"
    ms = 1000
    
    def elapsed(self):
        elap = datetime.now() - self.start
        return elap.seconds % 60
    
class Hundredths(TimeUnit):
    x = 210
    color = "orange"
    ms = 300
    dur = 0.2
    
    def elapsed(self):
        elap = datetime.now() - self.start
        return int(elap.microseconds/10000)

class Timer:
    def __init__(self):
        
        self.demo = dual_canvas.DualCanvasWidget(width=820, height=820, font='italic 50px Courier')
        
        self.demo.text(75,114,":",color='black')
        self.demo.text(175,114,":",color='black')
        #self.min_txts = self.text_init('red', 10, "00")
        #self.sec_txts = self.text_init('blue', 100, "00")
        #self.hsec_txts = self.text_init('orange', 190, "00")
        self.minutes = Minutes(self.demo)
        self.seconds = Seconds(self.demo)
        self.hundredths = Hundredths(self.demo)
        
        self.demo.rect(0,0,300,100,color='green')
        self.demo.rect(0,200,300,100,color='green')
        
        self.start = None
        self.demo.js_init("""
            element.setTimeout = function(callback, ms) {
                window.setTimeout(callback, ms);
            };
        """)
        self.demo.fit()
        self.widget = self.demo
    
    #def text_init(self,color, x, text_ini):
    #    txt1 = self.demo.text(x,10,text_ini,color=color, name=True)
    #    txt2 = self.demo.text(x,110,text_ini,color=color, name=True)
    #    txt3 = self.demo.text(x,210,text_ini,color=color, name=True)
    #    return [txt1, txt2, txt3]    
    
    def start_timing(self):
        
        self.minutes.repeat_timing()
        self.seconds.repeat_timing()
        self.hundredths.repeat_timing()
        #from datetime import datetime
        
        #self.start = datetime.now()
        #self.repeat_time('hsec', 300)
        #self.repeat_time('sec', 1000)
        #self.repeat_time('min', 60000)
    
    #def repeat_time(self, unit, ms):
    #    '''
    #    unit = 'min', 'sec', 'hsec'
    #    '''
    #    self.timing(unit)
    #    self.demo.element.setTimeout(lambda: self.repeat_time(unit, ms), ms)
    """
    def timing(self, unit):
        '''
        unit = 'min', 'sec', 'hsec'
        '''
        from datetime import datetime
        elap = datetime.now() - self.start
        sec = elap.seconds
        minute = int(sec/60)
        hsec = int(elap.microseconds/10000)
        
        [a, b, c] = [0,0,0]
        dur = 1
        if unit=='min':
            [a, b, c] = self.min_txts
            dur = 0.6
            self.txt_transition(a, b, c, dur, minute)
            self.min_txts = [c, a, b]
        elif unit=='sec':
            [a, b, c] = self.sec_txts
            dur = 0.6
            txt_sec = sec%60
            self.txt_transition(a, b, c, dur, txt_sec)
            self.sec_txts = [c, a, b]
        else:
            [a, b, c] = self.hsec_txts
            dur = 0.2
            self.txt_transition(a, b, c, dur, hsec)
            self.hsec_txts = [c, a, b]

    def txt_transition(self, a, b, c, seconds_duration, txt):
        b.transition(y=110, seconds_duration=seconds_duration, text='%02d' %txt)
        a.change(y=10)
        c.transition(y=210,seconds_duration=seconds_duration)"""

In [ ]:
t = Timer()
t.widget.debugging_display()

In [ ]:
t.start_timing()